In [ ]:
"""
This example demonstrates the impact of the C band hail reflectivity 
(Brook et al. 2023 https://arxiv.org/abs/2306.12016) using overlapping C and S band radars
for the Sydney 20-12-2018 hailstorm
"""

In [1]:
import os
from urllib import request
from matplotlib import pyplot as plt
import pyart
from pyhail import mesh_ppi


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



/g/data/en0/jss548/miniconda3/envs/openradar/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/g/data/en0/jss548/miniconda3/envs/openradar/lib/python3.8/site-packages/pyhail-2.0.1-py3.8.egg/pyhail/mesh_ppi.py:57: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/g/data/en0/jss548/miniconda3/envs/openradar/lib/python3.8/site-packages/pyhail-2.0.1-py3.8.egg/pyhail/mesh_ppi.py:57: SyntaxWarning: "is not" with a literal. Did you mean "!="?


In [2]:
#config
cband_radar_url  = 'https://pyhail-public.s3.ap-southeast-2.amazonaws.com/54_20181220_042503.pvol.h5'
sband_radar_url  = 'https://pyhail-public.s3.ap-southeast-2.amazonaws.com/71_20181220_052428.pvol.h5'
cband_refl_offset = -5.08 #dBZ
sband_refl_offset = 1.51 #dBZ
freezing_level = 3000 #m
minus20_level = 5000 #m
max_lat = -33.5
min_lat = -34.2
max_lon = 150.8
min_lon = 151.5
n_grid_lines = 5

In [3]:
def _create_ppi_plot(subplot_idx, fieldname, tilt, vmin, vmax, display, cmap=None, title=None, subplot_ncols=2, subplot_nrows=2):
    """
    subplot creation function
    """
    #create subplot axis
    ax=plt.subplot(subplot_ncols, subplot_nrows, subplot_idx, projection=ccrs.PlateCarree())
    if cmap is None:
        display.plot_ppi_map(fieldname,tilt, vmin=vmin, vmax=vmax, resolution = '10m', ax=ax,
                                max_lat = max_lat, min_lat =min_lat, min_lon = min_lon, max_lon = max_lon,
                                lat_lines = lat_lines, lon_lines = lon_lines)
    else:
        display.plot_ppi_map(fieldname,tilt, vmin=vmin, vmax=vmax, resolution = '10m', ax=ax,
                                max_lat = max_lat, min_lat =min_lat, min_lon = min_lon, max_lon = max_lon,
                                lat_lines = lat_lines, lon_lines = lon_lines,
                                cmap=cmap)
    #add title
    if title is not None:
        plt.set_title(title)

In [4]:
#download example data
tmp_path    = '/tmp/pyhail_example_data'
if not os.path.exists(tmp_path):
    os.makedirs(tmp_path)
cband_radar_ffn    = f'{tmp_path}/54_20181220_042503.pvol.h5'
sband_radar_ffn    = f'{tmp_path}/71_20181220_052428.pvol.h5'
request.urlretrieve(cband_radar_url, cband_radar_ffn)
request.urlretrieve(sband_radar_url, sband_radar_ffn)

#read example data
cband_radar = pyart.aux_io.read_odim_h5(cband_radar_ffn)
sband_radar = pyart.aux_io.read_odim_h5(sband_radar_ffn)

#calibrate reflectivity
cband_cal_refl = cband_radar.fields['reflectivity_horizontal']['data'] - cband_refl_offset
cband_radar.add_field_like('reflectivity_horizontal', 'corrected_reflectivity', cband_cal_refl)
sband_cal_refl = sband_radar.fields['reflectivity_horizontal']['data'] - sband_refl_offset
sband_radar.add_field_like('reflectivity_horizontal', 'corrected_reflectivity', sband_cal_refl)

#run mesh retrievals
meshppi_cband_fields = mesh_ppi.main(cband_radar, 'corrected_reflectivity', levels=[freezing_level, minus20_level], min_range=10, max_range=150,
                 mesh_method='mh2019_75', radar_band='C', correct_cband_refl=True)
meshppi_cband__uncorr_fields = mesh_ppi.main(cband_radar, 'corrected_reflectivity', levels=[freezing_level, minus20_level], min_range=10, max_range=150,
                 mesh_method='mh2019_75', radar_band='C', correct_cband_refl=False)
meshppi_sband_fields = mesh_ppi.main(sband_radar, 'corrected_reflectivity', levels=[freezing_level, minus20_level], min_range=10, max_range=150,
                 mesh_method='mh2019_75', radar_band='S', correct_cband_refl=True)

#add to a radar object
radar = cband_radar
radar.add_field('cband_mesh', meshppi_cband_fields, replace_existing=True)
radar.add_field('cband_uncorr_mesh', meshppi_cband__uncorr_fields, replace_existing=True)
radar.add_field('sband_mesh', meshppi_sband_fields, replace_existing=True)

#plot
fig = plt.figure(figsize=[10,10])
display = pyart.graph.RadarMapDisplay(radar)
lat_lines = np.linspace(min_lat, max_lat, n_grid_lines)
lon_lines = np.linspace(min_lon, max_lon, n_grid_lines)
_create_ppi_plot(0, 'cband_mesh', 0, 0, 50, cmap='hot_r')
_create_ppi_plot(1, 'cband_uncorr_mesh', 0, 0, 50, cmap='hot_r')
_create_ppi_plot(2, 'sband_mesh', 0, 0, 50, cmap='hot_r')

ValueError: radar_band must be a string of value C or S